**Problem 1.1** In this problem we implement the least-squares fusion technique in (1.60) in code to verify empirically verify the benefit of data fusion. You are free to choose any unspecified parameters.

**(a)** Generate local data sets $\{ h_{k, n}, \gamma_{k, n} \}_{n=1}^N$ following the statistical model of Example 1.4.

**Solution.** We begin by importing some standard packages which will be useful throughout this exercise:



In [21]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

We begin by setting the variance parameters $\sigma_w^2, \sigma_h^2, \sigma_v^2$, network size $K$, sample size $N$ and dimension $M$. Then generate a realization of the weight vector $w$ by sampling once from the normal distribution $\mathcal{N}(0, \sigma_w^2 I_M)$. For each agent $k$, we sample $N$ times from $\mathcal{N}(0, \sigma_h^2 I_M)$ to generate $\{ h_{k, n} \}_{n=1}^N$, and $N$ times from $\mathcal{N}(0, \sigma_v^2)$ to generate $\{ v_{k, n} \}_{n=1}^N$. $\{ \gamma_{k, n} \}_{n=1}^N$ are then generated according to the linear model (1.28). We store data sets in matrices for compact coding.

In [22]:
sigma_w = 1
sigma_h = 1
sigma_v = 1

rho = np.square(np.true_divide(sigma_v, sigma_w))

K = 1000
N = 1
M = 10

w = np.random.multivariate_normal(np.zeros(M), np.square(sigma_w)*np.eye(M))

h = np.zeros((M, N, K))
v = np.zeros((N, K))
gamma = np.zeros((N, K))
for k in range(K):
    h[:, :, k] = np.random.multivariate_normal(np.zeros(M), np.square(sigma_h)*np.eye(M), N).T
    v[:, k] = np.random.normal(0, np.square(sigma_v), N).T
    gamma[:, k] = np.matmul(h[:, :, k].T, w) + v[:, k]

**(b)** Compute $w_k^{\star}$ for each agent along with the globally optimal model $w^{\star}$.

**Solution.** We make use of relations (1.53) and (1.56). To this end, we need to compute $H_k$ and $d_k$ as well as $H$ and $d$.

In [23]:
H_k = np.zeros((M, M, K))
d_k = np.zeros((M, K))

H = np.zeros((M, M))
d = np.zeros(M)

w_k_star = np.zeros((M, K))

for k in range(K):
    for n in range(N):
        H_k[:, :, k] += np.outer(h[:, n, k], h[:, n, k])
        d_k[:, k] += gamma[n, k]*h[:, n, k]
    w_k_star[:, k] = np.linalg.solve(H_k[:, :, k] + rho*np.eye(M), d_k[:, k])

    H += H_k[:, :, k]
    d += d_k[:, k]

w_star = np.linalg.solve(H + rho*np.eye(M), d)


**(c)** Generate a new independent test set $\left\{\widetilde{h}_{n}, \widetilde{\gamma}_{n}\right\}_{n=1}^{\widetilde{N}}$ and evaluate prediction performance of local models:
    $$\frac{1}{\widetilde{N}} \sum_{n=1}^{\widetilde{N}} (\widetilde{\gamma}_n - \widetilde{h}_n^{\mathsf{T}} w_k^{\star})^2$$
  as well as of the performance of the global model:
    $$\frac{1}{\widetilde{N}} \sum_{n=1}^{\widetilde{N}} (\widetilde{\gamma}_n - \widetilde{h}_n^{\mathsf{T}} w^{\star})^2$$

**Solution.** We generate the test sets by sampling in the same manner as when we generated the training set.

In [24]:
N_tilde = 10000

h_tilde = np.random.multivariate_normal(np.zeros(M), np.square(sigma_h)*np.eye(M), N_tilde).T
v_tilde = np.random.normal(0, np.square(sigma_v), N_tilde).T
gamma_tilde = np.matmul(h_tilde.T, w) + v_tilde

Predictions then are computed as $\widetilde{h}_n^{\mathsf{T}} w_{k}^{\star}$ and $\widetilde{h}_n^{\mathsf{T}} w^{\star}$. We then compute the average mean-square prediction error of the local models with the mean-square prediction error of the global model.

In [25]:
gamma_predicted_k = np.zeros((N_tilde, K))
mse_k = np.zeros(K)
for k in range(K):
    gamma_predicted_k[:, k] = np.matmul(h_tilde.T, w_k_star[:, k])
    mse_k[k] = np.true_divide(np.square(np.linalg.norm(gamma_tilde - gamma_predicted_k[:, k])), N_tilde)
mse_k_ave = np.mean(mse_k)

gamma_predicted_glob = np.matmul(h_tilde.T, w_star)
mse_glob = np.true_divide(np.square(np.linalg.norm(gamma_tilde - gamma_predicted_glob)), N_tilde)

gamma_predicted_true = np.matmul(h_tilde.T, w)
mse_true = np.true_divide(np.square(np.linalg.norm(gamma_tilde - gamma_predicted_true)), N_tilde)

print('The average MSE of local models is:', mse_k_ave)
print('The MSE of the global model is:', mse_glob)
print('The MSE of the correct model is:', mse_true)

The average MSE of local models is: 3.9915414125506894
The MSE of the global model is: 0.9937291054620632
The MSE of the correct model is: 0.9861630700100513
